**This problem set is due Wednesday, September 22, 2021 at 11:59 pm. Please plan ahead and submit your work on time.**

Import the modules needed for this exercise (make sure you execute the cell below by clicking on it and pressing Shift-Enter)

In [1]:
%load_ext autoreload
%autoreload 2
from utils import test_ok, check_expanded_states, check_expanded_nodes
from search_classes import SearchNode, Path

# Problem Set 2: Solving the 8 Puzzle Problem with Search


1. [State representation in the 8 Puzzle Problem](#state_representation)
    1. [Expanding the puzzle state (25 points)](#state_expansion)
2. [Simple Search](#simple_search)
    1. [Completing the `PuzzleProblem` class (10 points)](#puzzle_problem_class)
    2. [Implementing Breath First Search (45 points)](#bfs_implementation)
    3. [Breadth First Search vs Depth First Search (20 points)](#bfs_vs_dfs)


In this Problem Set you will implement Breath First Search and use it to solve the [8 Puzzle Problem](https://en.wikipedia.org/wiki/15_puzzle).

The puzzle consists on a 3x3 grid with 8 numbered tiles and a missing tile. The objective consists on sliding the tiles around until all the numbered tiles are ordered and the missing tile stays at the lower right cell of the grid.

<img src="puzzle8.png"/>




## <a name="state_representation"></a>State representation in the 8 Puzzle Problem

To make things simple, we are giving you a possible state representation for the 8-puzzle problem.

We'll represent a given state of the puzzle by a tuple of three internal tuples. Each internal tuple represents a row of the puzzle. The missing tile is represented by $0$.

For example, the puzzle state below:

<img src="example_state.png"/>

is represented by `((1, 2, 3), (8, 0, 4), (7, 6, 5))`.

Below, we are giving you some code to print a puzzle state:

In [2]:
# (this cell will be overwritten)
def print_state(state):
    print("+"+ "-"*5+"+")
    for l in state:
        print("|"+ " ".join([str(el) if el!=0 else " " for el in l]) +"|")
    print("+"+ "-"*5+"+")

example_state = ((1, 2, 3), (8, 0, 4), (7, 6, 5))

print("%s state represents puzzle state: " % (example_state,))
print_state(example_state)

((1, 2, 3), (8, 0, 4), (7, 6, 5)) state represents puzzle state: 
+-----+
|1 2 3|
|8   4|
|7 6 5|
+-----+


### <a name="state_expansion"></a>Expanding a puzzle state (25 points)

In order to find a solution to the state, we need to define the states we can reach from a given state.

This corresponds to the possible moves of the missing tile (at most up, down, left and right).

Implement the function `expand_state(state)` that returns a `list` of the states that can be reached from the given `state`.

For example, for state `((0, 1, 3), (4, 2, 5), (7, 8, 6))`, the function `expand_state` should return the following list (two moves are feasible):

```
[((4, 1, 3), (0, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6))]
```

The neighbour states of state:

```
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+
```

are:

```
+-----+
|4 1 3|
|  2 5|
|7 8 6|
+-----+
***
+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+
```

<div class="alert alert-info">
Implement the function `expand_state(state)` below.
</div>


In [3]:
def expand_state(state):
    # YOUR CODE HERE
    nullspace=[]
    leftmove=[]
    upmove=[]
    downmove=[]
    rightmove=[]
    final=[]
#if the 0 is in the top row it can go down, left or right; if its on the bottom it can go up left or right
    for i in range(0,3):
        for j in range(0,3):
            if state[i][j]==0:
                nullspace.append(i)
                nullspace.append(j)

    if nullspace[0]==1:
        if nullspace[1]==1:
            leftmove=(state[0],(state[1][1], state[1][0], state[1][2]),state[2])
            rightmove=(state[0],(state[1][0], state[1][2], state[1][1]),state[2])
            upmove=((state[0][0], state[1][1], state[0][2]),(state[1][0],state[0][1],state[1][2]),state[2])
            downmove=(state[0],(state[1][0],state[2][1],state[1][2]),(state[2][0], state[1][1], state[2][2]))
            #print_state(downmove)
            #print_state(leftmove)
           # print_state(rightmove)
            #print_state(upmove)
            final=[downmove, upmove,rightmove,leftmove]
        if nullspace[1]==0:
            rightmove=(state[0],(state[1][1], state[1][0], state[1][2]),state[2])
            upmove=((state[1][0], state[0][1], state[0][2]),(state[0][0],state[1][1],state[1][2]),state[2])
            downmove=(state[0],(state[2][0],state[1][1],state[1][2]),(state[1][0], state[2][1], state[2][2]))
            #print_state(rightmove)
            #print_state(upmove)
            #print_state(downmove)
            final=[rightmove,upmove,downmove]
        if nullspace[1]==2:
            upmove=((state[0][0], state[0][1], state[1][2]),(state[1][0],state[1][1],state[0][2]),state[2])
            downmove=(state[0],(state[1][0],state[1][1],state[2][2]),(state[2][0], state[2][1], state[1][2]))
            leftmove=(state[0],(state[1][0], state[1][2], state[1][1]),state[2])
            #print_state(downmove)
            #print_state(leftmove)
            #print_state(upmove)
            final=[upmove,downmove]
    if nullspace[0]==0:
        if nullspace[1]==1:
            downmove=((state[0][0],state[1][1],state[0][2]),(state[1][0],state[0][1],state[1][2]),state[2])
            leftmove=((state[0][1],state[0][0],state[0][2]),state[1],state[2])
            rightmove=((state[0][0],state[0][2],state[0][1]),state[1],state[2])
            #print_state(downmove)
            #print_state(leftmove)
            #print_state(rightmove)
            final=[downmove,leftmove,rightmove]
        if nullspace[1]==0:
            rightmove=((state[0][1],state[0][0],state[0][2]),state[1],state[2])
            #print_state(rightmove)
            downmove=((state[1][0],state[0][1],state[0][2]),(state[0][0],state[1][1],state[1][2]),state[2])
            #print_state(downmove)
            final=[downmove,rightmove]
        if nullspace[1]==2:
            leftmove=((state[0][0],state[0][2],state[0][1]),state[1],state[2])
            #print_state(leftmove)
            downmove=((state[0][0],state[0][1],state[1][2]),(state[1][0],state[1][1],state[0][2]),state[2])
           #print_state(downmove)
            final=(downmove,leftmove)
    if nullspace[0]==2:
        if nullspace[1]==1:
            leftmove=(state[0],state[1],(state[2][1],state[2][0],state[2][2]))
            rightmove=(state[0],state[1],(state[2][0],state[2][2],state[2][1]))
            upmove=(state[0],(state[1][0],state[2][1],state[1][2]),(state[2][0],state[1][1],state[2][2]))
           # print_state(leftmove)
            #print_state(rightmove)
            #print_state(upmove)
            final=[leftmove,rightmove,upmove]
        if nullspace[1]==0:
            rightmove=(state[0],state[1],(state[2][1],state[2][0],state[2][2]))
            #print_state(rightmove)
            upmove=(state[0],(state[2][0],state[1][1],state[1][2]),(state[1][0],state[2][1],state[2][2]))
            #print_state(upmove)
            final=[upmove,rightmove]
        
        if nullspace[1]==2:
            leftmove=(state[0],state[1],(state[2][0],state[2][2],state[2][1]))
            #print_state(leftmove)
            upmove=(state[0],(state[1][0],state[1][1],state[2][2]),(state[2][0],state[2][1],state[1][2]))
            #print_state(upmove)
            final=[upmove,leftmove]
    return final
    raise NotImplementedError()

In [4]:
"""Check the expand state function"""
from nose.tools import assert_equal, ok_

check_expanded_states(expand_state(((0, 1, 3), (4, 2, 5), (7, 8, 6))),
                     [((4, 1, 3), (0, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6))])

check_expanded_states(expand_state(((1, 2, 3), (8, 0, 4), (7, 6, 5))),
                     [((1, 2, 3), (8, 6, 4), (7, 0, 5)),
                     ((1, 0, 3), (8, 2, 4), (7, 6, 5)),
                     ((1, 2, 3), (8, 4, 0), (7, 6, 5)),
                     ((1, 2, 3), (0, 8, 4), (7, 6, 5))])
test_ok()

Test passed!!

## <a name="simple_search"></a>Simple Search

Now you will implement Simple Search, as seen in class, to solve the 8 Puzzle Problem.

We are giving you the class `SearchNode` defined in `search_classes.py`. This class represents a search node in the search tree.

You can create a `SearchNode` by giving it the state it represents and its `SearchNode` parent (or None if it's the root element in the tree). Below is an example of the Serach Node class being used:

In [5]:
# Execute this example code
root_node = SearchNode(((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent_node=None)
children_node = SearchNode(((4, 1, 3), (0, 2, 5), (7, 8, 6)),
                            parent_node=root_node)
print("Root node: %s" % root_node)
print("Children node: %s" % children_node)

Root node: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>
Children node: <SearchNode: state: ((4, 1, 3), (0, 2, 5), (7, 8, 6)), parent: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>>


We also give you the `Path` class, that takes a `SearchNode` and computes the state path from the initial state in the root of the tree to the state of the given `SearchNode`:

In [6]:
# Execute this example code
example_path = Path(children_node)
print(example_path.path)
print(example_path)
print(children_node)
print("Path of %d states is: %s" % (len(example_path.path), example_path.path))

[((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((4, 1, 3), (0, 2, 5), (7, 8, 6))]
Path of length 2: [((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((4, 1, 3), (0, 2, 5), (7, 8, 6))]
<SearchNode: state: ((4, 1, 3), (0, 2, 5), (7, 8, 6)), parent: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>>
Path of 2 states is: [((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((4, 1, 3), (0, 2, 5), (7, 8, 6))]


<div class="alert alert-warning">
You will want to look at the `SearchNode` and `Path` definitions in the included **`search_classes.py`** file, as you will need to know what useful properties you can use for the next questions.
</div>

In [7]:
class PuzzleProblem(object):
    
    """Class that represents the puzzle search problem."""
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
    def test_goal(self, state):
        return self.goal == state
    def expand_node(self, search_node):
        """Return a list of SearchNodes, having the correct state and parent node."""
        # YOUR CODE HERE
        root_node = SearchNode(search_node.state, parent_node=None)
        expanded=[]
        expanded=expand_state(search_node.state)
        expanded_sn=[]
        for i in range(0,len(expanded)):
            expanded_sn.append(SearchNode(expanded[i],parent_node=search_node))
        
        return expanded_sn
        raise NotImplementedError()
        return expanded_sn

In [8]:
"""Check the implementation of PuzzleProblem"""
state_test = ((1, 2, 3), (8, 0, 4), (7, 6, 5))
problem_test = PuzzleProblem(state_test, None)
node_test = SearchNode(state_test, None)
check_expanded_nodes(problem_test.expand_node(node_test),
                     node_test,
                     [((1, 2, 3), (8, 6, 4), (7, 0, 5)),
                     ((1, 0, 3), (8, 2, 4), (7, 6, 5)),
                     ((1, 2, 3), (8, 4, 0), (7, 6, 5)),
                     ((1, 2, 3), (0, 8, 4), (7, 6, 5))])
test_ok()

Test passed!!

### <a name="bfs_implementation"></a>Implement Breadth First Search (45 points)

Finally, you'll implement *Breath First Search*.

Implement the function `breadth_first_search(search_problem)` that takes an instance of the `PuzzleProblem` class that we defined above and returns a tuple of three elements, in the following order:

1. If BFS finds a solution, an instance of the `Path` class containing that solution. If it doesn't, it should return `None` as the first element of the tuple.
2. The number of visited nodes
3. The maximum size of the queue

You should use a **visited list**, as otherwise the number of explored states in this problem will be large.

Also, think that instead of implementing Breadth First Search directly, you could very easily implement a generic *simple_search* function that takes as one of the parameters a function to insert an element in the queue and use this function to implement breath first search, and also depth first search very easily. However, you are not required to do this.

<div class="alert alert-info">
Implement `breadth_first_search(search_problem)` below.
</div>



In [9]:
def breadth_first_search(search_problem):
    """This function should take a PuzzleProblem instance
    and return a 3 element tuple as described above."""
# NEED TO UPDATE TO MAKE SURE WE ARE NOT EXECUTING THE SEARCH IF A STATE IS IN THE VISITED LIST
    visited=[]
    w=[]
    current_state=[]
    current_sn=[]
    start_sn=[]
    queue=[]
    sol=None
    goal=goal_state
    next_sn=[]
    maxq=1
    maxsteps=1000
    next_statetoexpand=[]

    start_sn=SearchNode(start_state, None)

    queue.append(start_sn)
    first_sn=queue.pop(0)
    visited.append(start_sn.state)
    visitlength=(len(visited))
    
    if first_sn.state==goal:
        sol=Path(current_sn)
        print("you've reached a solution")
        return (sol,1, 1)
    next_sn=search_problem.expand_node(SearchNode(start_state,None))
    for q in range(0, len(next_sn)):
        queue.append(next_sn[q])
    if len(queue)>maxq:
        maxq=len(queue)
    
    for w in range(1,maxsteps):
        if len(queue)>maxq:
            maxq=len(queue)
        current_sn=queue.pop(0)
        if current_sn.state in visited:
            print ("You've been here before; your search has returned you to a visited state")
            continue
        visited.append(current_sn.state)
        visitlength=(len(visited))
        if current_sn.state==goal:
            sol=Path(current_sn)
            print("You've reached a solution")
            return (sol,visitlength, maxq)
        next_sn=search_problem.expand_node(current_sn)
        for q in range(0, len(next_sn)):
            if next_sn[q].state in visited:
                continue
            queue.append(next_sn[q])
        
        
    return (sol,visitlength, maxq)
        
    raise NotImplementedError()



In [10]:
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))
goal_state = ((1,2,3),(4,5,6),(7,8,0))
problem = PuzzleProblem(start_state, goal_state)

sol, num_visited, max_q = breadth_first_search(problem)
print("the solution path is ", sol)
print("the number of nodes visited is: ", num_visited)
print("the max size of the queue is: ", max_q)

You've reached a solution
the solution path is  Path of length 5: [((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6)), ((1, 2, 3), (4, 0, 5), (7, 8, 6)), ((1, 2, 3), (4, 5, 0), (7, 8, 6)), ((1, 2, 3), (4, 5, 6), (7, 8, 0))]
the number of nodes visited is:  27
the max size of the queue is:  17


### Solve the Puzzle Problem using BFS

Finally, let's use your Breath First Search implementation to solve the 8 Puzzle Problem.
Execute the cell below. If your BFS implementation is correct, you should see the solution printed below.
Don't modify the cell below, as it will be overwritten by our software. If you want to experiment with different states, create new cells below.


In [11]:
# Solve the 8 Puzzle Problem from state:
# +-----+
# |  1 3|
# |4 2 5|
# |7 8 6|
# +-----+
# Don't modify this cell (contents will be overwritten by autograder)
# If you want to experiment with other states, try adding cells below.
# You can try with state: ((1, 8, 2), (0, 4, 3), (7, 6, 5)) for example.
# Remember that not all states have a solution. Try ((8, 1, 2), (0, 4, 3), (7, 6, 5)), for example.
# Be ready to wait, though!
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))
start_state = ((8, 1, 2), (0, 4, 3), (7, 6, 5))
goal_state = ((1,2,3),(4,5,6),(7,8,0))
problem = PuzzleProblem(start_state, goal_state)

sol, num_visited, max_q = breadth_first_search(problem)
if sol:
    print("Solution found!\n%d states in the solution (%d moves)\n%d states explored.\n%d maximum queue" \
          %(len(sol.path), len(sol.path)-1, num_visited,max_q))
    print("Solution: ")
    for s in sol.path:
        print_state(s)
        print("\n**\n")
else:
    print("No solution after exploring %d states with max q of %d" %(num_visited, max_q))

You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned you to a visited state
You've been here before; your search has returned y

### <a name="bfs_vs_dfs"></a> Breadth First Search vs Depth First Search (20 points)

Have you tried using DFS to solve this problem?

Would DFS be a better choice for this problem? What benefits does BFS have over DFS in this problem? Please explain in the cell below (double click on the cell below, remove the text and type your answer). This is a qualitative question, you are not required to give numerical comparisons.

I did not try and use DFS for this problem yet. mostly because of this conclusion: I think that BFS is a better choice for this problem. For this problem, BFS is guaranteed to find a path if one exists and that path will also be the shortest path. On the other hand, DFS typically only finds a solution for finite graphs; DFS will also likely take more time to solve this problem. Since the b (bushiness factor) for this problem is small, even though BFS will require more memory to solve the problem, BFS will do it more quickly that DFS! 